In [ ]:
import numpy as np
import pandas as pd

np.random.seed(1993)

In [ ]:
a = np.array([1,2,3])
b = np.array([5,6,7])
(a + b) / 2

In [ ]:
train = pd.read_json("../input/train.json")

In [ ]:
train.head()

In [ ]:
len(train.band_1[0])

In [ ]:
train.shape

In [ ]:
train = train[train['inc_angle'] != 'na']

In [ ]:
train_label = train.is_iceberg
train_label = np.array(train_label, dtype='float')

In [ ]:
train_label

In [ ]:
train_data = train.drop(columns=['is_iceberg', 'id'])

In [ ]:
train_data = train_data[train_data['inc_angle'] != 'na']

In [ ]:
train_data.shape

In [ ]:
rows = train_data.shape[0]

In [ ]:
train_data.head()

In [ ]:
train_data.tail()

In [ ]:
angle = np.array([x[2] for x in train_data.values])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_angle = MinMaxScaler(feature_range=(1,2))

In [ ]:
angle = scaler_angle.fit_transform(np.matrix(angle).T).reshape(rows)

In [ ]:
angle

In [ ]:
len(angle), len(train_data.values)

In [ ]:
#data_bands = np.array([np.concatenate((np.array(x[0]), np.array(x[1]))) for x in train_data.values])
data_bands = np.array([np.concatenate(((np.array(x[0]) * a), (np.array(x[1]) * a))) for x, a in zip(train_data.values, angle)])

In [ ]:
data_bands.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
data_bands = scaler.fit_transform(data_bands)

In [ ]:
data_bands[:3]

In [ ]:
imgs_bands = np.reshape(data_bands, (rows, 75, 75, 2))

In [ ]:
print(imgs_bands.shape, train_label.shape)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X, X_test, y, y_test = train_test_split(imgs_bands, train_label, test_size=0.60, stratify=train_label)

In [ ]:
from keras import layers
from keras import models
from keras.constraints import maxnorm

def create_model(optimizer='RMSprop', init_mode='lecun_uniform', activation='relu', map_conv=[(8, 8), (4, 4), (2, 2)], out_layers=[32,64,64,64], dropout=.2, weight_constraint=4):
    model = models.Sequential()
    model.add(layers.Conv2D(out_layers[0], map_conv[0], activation=activation, input_shape=(75, 75, 2)))    
    model.add(layers.MaxPooling2D((2, 2)))    
    model.add(layers.Conv2D(out_layers[1], map_conv[1], activation=activation))
    model.add(layers.MaxPooling2D((2, 2)))    
    model.add(layers.Conv2D(out_layers[2], map_conv[2], activation=activation, kernel_constraint=maxnorm(weight_constraint)))    
    model.add(layers.Dropout(dropout))
    model.add(layers.Flatten())
    model.add(layers.Dense(out_layers[3], activation=activation, kernel_constraint=maxnorm(weight_constraint)))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = create_model()
model.fit(X, y, epochs=20, batch_size=8, validation_data=(X_test, y_test))

In [ ]:
from keras.models import model_from_json
import os

In [ ]:
model_json = model.to_json()
with open('./model/covnet-v3.json', 'w') as json_file:
    json_file.write(model_json)

In [ ]:
model.save_weights('./model/covnet-v3.h5')

In [ ]:
test = pd.read_json('../input/test.json')

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
test_bands = np.array([np.concatenate((np.array(x[0]), np.array(x[1]))) for x in test.values])

In [ ]:
test_bands = scaler.transform(test_bands)

In [ ]:
test_bands.shape

In [ ]:
test_bands = np.reshape(test_bands, (8424, 75, 75, 2))

In [ ]:
ids, is_iceberg = [], []

In [ ]:
for row in range(len(test_bands)):
    img = test_bands[row]
    ids.append(test.id[row])
    pred = model.predict(np.array([img]))
    is_iceberg.append(pred[0][0])

print(ids[:10], is_iceberg[:10])

In [ ]:
df_result = pd.DataFrame({'id': ids,'is_iceberg': is_iceberg})

In [ ]:
df_result.head()

In [ ]:
test.tail()

In [ ]:
df_result.to_csv('submission_v3.csv', encoding='utf-8', float_format='%.16f', index=False)

In [ ]:
df_result.is_iceberg.mean()

In [ ]:
df_result.is_iceberg.std()